# NASA Asteroid Classification

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.decomposition import PCA
from scipy.stats import entropy

# load the dataset
from collections import Counter
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier

# Functions

In [2]:
def run_all_models(X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded):
    models = {
        "LogisticRegression": LogisticRegression(),
        "SVC": SVC(),
        "DecisionTreeClassifier": DecisionTreeClassifier(random_state=42),
        "RandomForestClassifier": RandomForestClassifier(random_state=42),
        "KNeighborsClassifier": KNeighborsClassifier(),
        "GradientBoostingClassifier": GradientBoostingClassifier(random_state=42),
        "GaussianNB": GaussianNB(),
        "LDA":LinearDiscriminantAnalysis(),
        "GPC":GaussianProcessClassifier()
    }

    # Create an empty DataFrame to store model performance
    model_performance = []

    for name, model in models.items():
        model.fit(X_train_scaled, y_train_encoded)
        predictions = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test_encoded, predictions)
        model_performance.append({
            "Model": name,
            "Accuracy": accuracy
        })

    # For the Sequential model
    sequential_model = Sequential()
    sequential_model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
    sequential_model.add(Dense(32, activation='relu'))
    sequential_model.add(Dense(1, activation='sigmoid'))
    sequential_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    sequential_model.fit(X_train_scaled, y_train_encoded, epochs=50, batch_size=10, verbose=0)
    loss, accuracy = sequential_model.evaluate(X_test_scaled, y_test_encoded)
    model_performance.append({
        "Model": "Sequential",
        "Accuracy": accuracy
    })

    # Convert the model_performance to a DataFrame
    return pd.DataFrame(model_performance)

def automate_pca(pca_components, X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded):
  pca = PCA(n_components=pca_components)
  X_train_pca = pca.fit_transform(X_train_scaled)
  X_test_pca = pca.transform(X_test_scaled)
  
  models = run_all_models(X_train_pca, X_test_pca, y_train_encoded, y_test_encoded)
  print(models)
  
def labels_to_probabilities(y):
    values, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    return probabilities

def calculate_entropy(df:pd.DataFrame)-> pd.DataFrame:
  """_summary_

  Args:
      df (pd.DataFrame): _description_

  Returns:
      pd.DataFrame: _description_
  """

  column_entropy_info = {}
  for col in df.columns:
    probabilities = labels_to_probabilities(df[col])
    entropy_value = entropy(probabilities, base=2)
    column_entropy_info[col] = {
          'entropy': entropy_value
      }

  return pd.DataFrame(column_entropy_info)

# EDA Analysis

In [3]:
df = pd.read_csv("nasa.csv")
df

Neo Reference ID     Name  Absolute Magnitude  Est Dia in KM(min)  \
0              3703080  3703080              21.600            0.127220   
1              3723955  3723955              21.300            0.146068   
2              2446862  2446862              20.300            0.231502   
3              3092506  3092506              27.400            0.008801   
4              3514799  3514799              21.600            0.127220   
...                ...      ...                 ...                 ...   
4682           3759007  3759007              23.900            0.044112   
4683           3759295  3759295              28.200            0.006089   
4684           3759714  3759714              22.700            0.076658   
4685           3759720  3759720              21.800            0.116026   
4686           3772978  3772978              19.109            0.400641   

      Est Dia in KM(max)  Est Dia in M(min)  Est Dia in M(max)  \
0               0.284472         127.219879         284.472297   
1               0.326618         146.067964         326.617897   
2               0.517654         231.502122         517.654482   
3               0.019681           8.801465          19.680675   
4               0.284472         127.219879         284.472297   
...                  ...                ...                ...   
4682            0.098637          44.111820          98.637028   
4683            0.013616           6.089126          13.615700   
4684            0.171412          76.657557         171.411509   
4685            0.259442         116.025908         259.441818   
4686            0.895860         400.640618         895.859655   

      Est Dia in Miles(min)  Est Dia in Miles(max)  Est Dia in Feet(min)  ...  \
0                  0.079051               0.176763            417.388066  ...   
1                  0.090762               0.202951            479.225620  ...   
2                  0.143849               0.321655            759.521423  ...   
3                  0.005469               0.012229             28.876199  ...   
4                  0.079051               0.176763            417.388066  ...   
...                     ...                    ...                   ...  ...   
4682               0.027410               0.061290            144.723824  ...   
4683               0.003784               0.008460             19.977449  ...   
4684               0.047633               0.106510            251.501180  ...   
4685               0.072095               0.161210            380.662441  ...   
4686               0.248946               0.556661           1314.437764  ...   

      Asc Node Longitude Orbital Period  Perihelion Distance  Perihelion Arg  \
0             314.373913     609.599786             0.808259       57.257470   
1             136.717242     425.869294             0.718200      313.091975   
2             259.475979     643.580228             0.950791      248.415038   
3              57.173266     514.082140             0.983902       18.707701   
4              84.629307     495.597821             0.967687      158.263596   
...                  ...            ...                  ...             ...   
4682          164.183305     457.179984             0.741558      276.395697   
4683          345.225230     407.185767             0.996434       42.111064   
4684           37.026468     690.054279             0.965760      274.692712   
4685          163.802910     662.048343             1.185467      180.346090   
4686          187.642183     653.679098             0.876110      222.436688   

      Aphelion Dist  Perihelion Time  Mean Anomaly  Mean Motion  Equinox  \
0          2.005764     2.458162e+06    264.837533     0.590551    J2000   
1          1.497352     2.457795e+06    173.741112     0.845330    J2000   
2          1.966857     2.458120e+06    292.893654     0.559371    J2000   
3          1.527904     2.457902e+06     68.741007     0.700277    J2000  

In [4]:
df["Equinox"].value_counts()

Equinox
J2000    4687
Name: count, dtype: int64

In [5]:
df.drop(["Equinox", "Neo Reference ID", "Name"], axis=1, inplace=True)

In [7]:
df.shape

(4687, 37)

In [9]:

length = len(df.columns)
target = df.values[:,-1]

counter = Counter(target)
for k,v in counter.items():
  per = v/len(target) * 100
  print("Class=%d, Count=%d, Percentage=%.3f%%" % (k,v,per))


Class=1, Count=755, Percentage=16.108%
Class=0, Count=3932, Percentage=83.892%


# Imbalanced Dataset

### We need to be careful regarding our approach due to this imbalance, high accuracy scores mean nothing with imbalanced datasets